<a href="https://colab.research.google.com/github/lhwong/airbnb-analytics/blob/main/paidout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>